In [21]:
%%capture
%pip install polars xlsx2csv openpyxl fastexcel
import polars as pl

from datetime import datetime, timedelta

In [124]:
server_components = pl.read_excel("data/program configurations.xlsx", sheet_name="server_specs")
rack_components = pl.read_excel("data/program configurations.xlsx", sheet_name="rack_specs")

quote_summary = pl.read_csv("data/quote_summaries.csv")
quote_details = pl.read_csv("data/quote_lines.csv")

In [125]:
print(server_components.head(2),
rack_components.head(2),
quote_details.head(2),
quote_summary.head(2))

shape: (2, 6)
┌──────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ Item ┆ Server A ┆ Server B ┆ Server C ┆ Server D ┆ Server E │
│ ---  ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str  ┆ i64      ┆ i64      ┆ i64      ┆ i64      ┆ i64      │
╞══════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ CPU  ┆ 2        ┆ 2        ┆ 2        ┆ 1        ┆ 1        │
│ GPU  ┆ 0        ┆ 4        ┆ 0        ┆ 0        ┆ 2        │
└──────┴──────────┴──────────┴──────────┴──────────┴──────────┘ shape: (2, 6)
┌─────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ Item    ┆ Server A ┆ Server B ┆ Server C ┆ Server D ┆ Server E │
│ ---     ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str     ┆ i64      ┆ i64      ┆ i64      ┆ i64      ┆ i64      │
╞═════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ SERVERS ┆ 12       ┆ 8        ┆ 14       ┆ 10       ┆ 10       │
│ TOR     ┆ 1        ┆ 2        ┆ 1        ┆ 1        ┆ 1 

In [126]:
def get_first_monday(year, month):
    first_day = datetime(year, month, 1)
    days_until_monday = (7 - first_day.weekday()) % 7
    return first_day + timedelta(days=days_until_monday)

def fct_valid_dates(df, year, month):
    first_monday = get_first_monday(year, month)
    month_25th = datetime(year, month, 25)
    
    # Filter data to include only dates between first Monday and 25th
    df_return = (
        df.with_columns(
            pl.col("quote_timestamp").str.to_datetime()
        )
        .filter(
            pl.col("quote_timestamp").is_between(first_monday, month_25th)
        )
    )
    
    return df_return

In [128]:
# Standardize column names in both DataFrames before joining
valid_quote_details = (
    fct_valid_dates(quote_details, year=2024, month=9)
    .filter(~((pl.col("Vendor") == "Vendor_7") & (pl.col("Program").is_in(["Program_C", "Program_E"]))))
)

valid_quote_summary = (
    fct_valid_dates(quote_summary, year=2024, month=9)
    .filter(~((pl.col("vendor") == "Vendor_7") & (pl.col("program").is_in(["Program_C", "Program_E"]))))
    .rename({"vendor": "Vendor", "program": "Program"})  # Standardize column names
)

# Aggregate line items to get total cost per quote
aggregated_details = (
    valid_quote_details
    .group_by(["Vendor", "Program", "quote_timestamp"])
    .agg(pl.sum("CPU").alias("total_cost"))  # Adjust aggregation to match your component of interest
)

# Join with summary and validate with a tolerance for floating-point comparison
validated_quotes = (
    aggregated_details
    .join(valid_quote_summary, on=["Vendor", "Program", "quote_timestamp"])
    .filter((pl.col("total_cost") - pl.col("reported_total_price")).abs() < 0.01)
)

# Combine server and rack components for total quantities across programs
component_quantities = (
    pl.concat([server_components, rack_components], how="vertical")
    .group_by("Item")
    .agg([
        pl.col(program).sum().alias(f"Program_{program}") 
        for program in ["Server A", "Server B", "Server C", "Server D", "Server E"]
    ])
)

# Calculate total cost for the latest quote received per vendor and program
latest_quotes = (
    validated_quotes
    .sort(by="quote_timestamp", descending=True)
    .group_by(["Vendor", "Program"])
    .agg(pl.first("reported_total_price").alias("total_server_rack_cost"))
)

# Calculate total cost for the first quote received per vendor and program
first_quotes = (
    validated_quotes
    .sort(by="quote_timestamp", descending=False)
    .group_by(["Vendor", "Program"])
    .agg(pl.first("reported_total_price").alias("total_server_rack_cost"))
)

# Identify minimum component prices per program and sum for best-in-class
best_in_class_pricing = (
    valid_quote_details
    .unpivot(
        index=["Vendor", "Program"],  # Equivalent to id_vars in melt
        on=["CPU", "GPU", "PSU", "TRAY", "TOR", "CHASSIS"]  # Equivalent to value_vars in melt
    )
    .rename({"variable": "component", "value": "price"})  # Rename columns for clarity
    .group_by(["Program", "component"])
    .agg(pl.min("price").alias("min_price"))
    .group_by("Program")
    .agg(pl.sum("min_price").alias("best_in_class_total_cost"))
)


valid_quote_details
valid_quote_summary
aggregated_details
validated_quotes
# component_quantities
# latest_quotes
# first_quotes
# best_in_class_pricing


Vendor,Program,quote_timestamp,total_cost,reported_total_price
str,str,datetime[μs],f64,f64


In [176]:
import polars as pl
from datetime import datetime, timedelta

# Helper function to get the first Monday of a given month
def get_first_monday(year, month):
    first_day = datetime(year, month, 1)
    days_until_monday = (7 - first_day.weekday()) % 7
    return first_day + timedelta(days=days_until_monday)

# Function to filter quotes within valid dates
def fct_valid_dates(df, year, month):
    first_monday = get_first_monday(year, month)
    month_25th = datetime(year, month, 25)
    print(f"First Monday: {first_monday}, Month 25th: {month_25th}")

    # Ensure 'quote_timestamp' is datetime
    df_return = (
        df.with_columns(
            pl.col("quote_timestamp").str.to_datetime()
        )
        .filter(
            pl.col("quote_timestamp").is_between(first_monday, month_25th)
        )
    )
    print("Filtered dates count:", df_return.shape[0])
    return df_return

# Load data
server_components = pl.read_excel("data/program configurations.xlsx", sheet_name="server_specs")
rack_components = pl.read_excel("data/program configurations.xlsx", sheet_name="rack_specs")
quote_summary = pl.read_csv("data/quote_summaries.csv")
quote_details = pl.read_csv("data/quote_lines.csv")

# Apply date filtering and program exclusion
valid_quote_details = (
    fct_valid_dates(quote_details, year=2024, month=9)
    .filter(~((pl.col("Vendor") == "Vendor_7") & (pl.col("Program").is_in(["Program_C", "Program_E"]))))
)

valid_quote_summary = (
    fct_valid_dates(quote_summary, year=2024, month=9)
    .filter(~((pl.col("vendor") == "Vendor_7") & (pl.col("program").is_in(["Program_C", "Program_E"]))))
    .rename({"vendor": "Vendor", "program": "Program"})
)

# Standardize date columns
valid_quote_details = valid_quote_details.with_columns(
    pl.col("quote_timestamp").dt.date().alias("quote_date")
)
valid_quote_summary = valid_quote_summary.with_columns(
    pl.col("quote_timestamp").dt.date().alias("quote_date")
)

# Unpivot server and rack component quantities correctly
server_quantities = server_components.unpivot(
    index=["Item"],
    on=["Server A", "Server B", "Server C", "Server D", "Server E"]
).with_columns(pl.col("variable").str.replace("Server ", "").alias("Program")).rename({"value": "Quantity"})

rack_quantities = rack_components.unpivot(
    index=["Item"],
    on=["Server A", "Server B", "Server C", "Server D", "Server E"]
).with_columns(pl.col("variable").str.replace("Server ", "").alias("Program")).rename({"value": "Quantity"})

# Combine server and rack quantities
component_quantities = pl.concat([server_quantities, rack_quantities])

# Unpivot valid_quote_details to reshape into long format
reshaped_quote_details = valid_quote_details.unpivot(
    index=["Vendor", "Program", "quote_date"],
    on=["CPU", "GPU", "RAM", "SSD", "HDD", "MOBO", "NIC", "PSU", "TRAY", "TOR", "CHASSIS"],
    variable_name="Item",
    value_name="Unit_Price"
)

# Merge quantities with reshaped quote details based on component names
detailed_with_quantities = (
    reshaped_quote_details
    .join(component_quantities, on=["Program", "Item"], how="inner")
    .with_columns((pl.col("Quantity") * pl.col("Unit_Price")).alias("extended_cost"))
)

# Calculate total cost using extended quantities
aggregated_details = (
    detailed_with_quantities
    .group_by(["Vendor", "Program", "quote_date"])
    .agg(pl.sum("extended_cost").alias("total_cost"))
)

# Join with valid_quote_summary and filter by cost difference
validated_quotes = (
    aggregated_details
    .join(valid_quote_summary, on=["Vendor", "Program", "quote_date"])
    .filter((pl.col("total_cost") - pl.col("reported_total_price")).abs() <= 5.0)
)

# Calculate latest and first quotes
latest_quotes = (
    validated_quotes
    .sort(by="quote_date", descending=True)
    .group_by(["Vendor", "Program"])
    .agg(pl.first("reported_total_price").alias("total_server_rack_cost"))
)

first_quotes = (
    validated_quotes
    .sort(by="quote_date", descending=False)
    .group_by(["Vendor", "Program"])
    .agg(pl.first("reported_total_price").alias("total_server_rack_cost"))
)

# Best-in-class pricing calculation
best_in_class_pricing = (
    reshaped_quote_details
    .group_by(["Program", "Item"])
    .agg(pl.min("Unit_Price").alias("min_price"))
    .group_by("Program")
    .agg(pl.sum("min_price").alias("best_in_class_total_cost"))
)

# Output final tables
print("Component Quantities Table:")
print(component_quantities)

print("Latest Quotes Table:")
print(latest_quotes)

print("First Quotes Table:")
print(first_quotes)

print("Best-in-Class Pricing Table:")
print(best_in_class_pricing)


First Monday: 2024-09-02 00:00:00, Month 25th: 2024-09-25 00:00:00
Filtered dates count: 164
First Monday: 2024-09-02 00:00:00, Month 25th: 2024-09-25 00:00:00
Filtered dates count: 164
Component Quantities Table:
shape: (60, 4)
┌─────────┬──────────┬──────────┬─────────┐
│ Item    ┆ variable ┆ Quantity ┆ Program │
│ ---     ┆ ---      ┆ ---      ┆ ---     │
│ str     ┆ str      ┆ i64      ┆ str     │
╞═════════╪══════════╪══════════╪═════════╡
│ CPU     ┆ Server A ┆ 2        ┆ A       │
│ GPU     ┆ Server A ┆ 0        ┆ A       │
│ RAM     ┆ Server A ┆ 4        ┆ A       │
│ SSD     ┆ Server A ┆ 1        ┆ A       │
│ HDD     ┆ Server A ┆ 0        ┆ A       │
│ …       ┆ …        ┆ …        ┆ …       │
│ TOR     ┆ Server D ┆ 1        ┆ D       │
│ CHASSIS ┆ Server D ┆ 1        ┆ D       │
│ SERVERS ┆ Server E ┆ 10       ┆ E       │
│ TOR     ┆ Server E ┆ 1        ┆ E       │
│ CHASSIS ┆ Server E ┆ 1        ┆ E       │
└─────────┴──────────┴──────────┴─────────┘
Latest Quotes Table:
sh

In [193]:
import polars as pl
from datetime import datetime, timedelta

# Helper function to get the first Monday of a given month
def get_first_monday(year, month):
    first_day = datetime(year, month, 1)
    days_until_monday = (7 - first_day.weekday()) % 7
    return first_day + timedelta(days=days_until_monday)

# Function to filter quotes within valid dates
def fct_valid_dates(df, year, month):
    first_monday = get_first_monday(year, month)
    month_25th = datetime(year, month, 25)
    return (
        df.with_columns(pl.col("quote_timestamp").str.to_datetime())
        .filter(pl.col("quote_timestamp").is_between(first_monday, month_25th))
    )

# Load data
server_components = pl.read_excel("data/program configurations.xlsx", sheet_name="server_specs")
rack_components = pl.read_excel("data/program configurations.xlsx", sheet_name="rack_specs")
quote_summary = pl.read_csv("data/quote_summaries.csv")
quote_details = pl.read_csv("data/quote_lines.csv")

# Step 1: Filter data based on spec requirements
# Apply date filter and exclude Vendor 7's quotes for Program C and Program E
valid_quote_details = (
    fct_valid_dates(quote_details, year=2024, month=9)
    .filter(~((pl.col("Vendor") == "Vendor_7") & (pl.col("Program").is_in(["Program_C", "Program_E"]))))
)

valid_quote_summary = (
    fct_valid_dates(quote_summary, year=2024, month=9)
    .filter(~((pl.col("vendor") == "Vendor_7") & (pl.col("program").is_in(["Program_C", "Program_E"]))))
    .rename({"vendor": "Vendor", "program": "Program"})
)

# Standardize date columns for easy matching
valid_quote_details = valid_quote_details.with_columns(pl.col("quote_timestamp").dt.date().alias("quote_date"))
valid_quote_summary = valid_quote_summary.with_columns(pl.col("quote_timestamp").dt.date().alias("quote_date"))

# Step 2: Create component quantities table by combining server and rack data
# Unpivot server and rack component quantities, renaming columns for consistency
server_quantities = server_components.melt(id_vars=["Item"], variable_name="Program", value_name="Quantity")
rack_quantities = rack_components.melt(id_vars=["Item"], variable_name="Program", value_name="Quantity")

# Combine server and rack quantities into a single DataFrame and clean Program names by removing 'Server '
component_quantities = (
    pl.concat([server_quantities, rack_quantities])
    .with_columns(pl.col("Program").str.replace("Server ", ""))
)

# Step 3: Reshape `valid_quote_details` for cost aggregation
# Unpivot to create long format for each component's unit price
reshaped_quote_details = valid_quote_details.melt(
    id_vars=["Vendor", "Program", "quote_date"],
    variable_name="Item",
    value_name="Unit_Price"
)

# Merge quantities with reshaped quote details based on Program and Item
detailed_with_quantities = (
    reshaped_quote_details
    .join(component_quantities, on=["Program", "Item"], how="inner")
    .with_columns((pl.col("Quantity") * pl.col("Unit_Price")).alias("extended_cost"))
)

# Step 4: Aggregate detailed costs and validate against summary totals
aggregated_details = (
    detailed_with_quantities
    .group_by(["Vendor", "Program", "quote_date"])
    .agg(pl.sum("extended_cost").alias("total_cost"))
)

# Join aggregated details with valid summary data and filter based on tolerance
validated_quotes = (
    aggregated_details
    .join(valid_quote_summary, on=["Vendor", "Program", "quote_date"])
    .filter((pl.col("total_cost") - pl.col("reported_total_price")).abs() <= 5.0)
)

# Calculate latest and first quotes
latest_quotes = (
    validated_quotes
    .sort(by="quote_date", descending=True)
    .group_by(["Vendor", "Program"])
    .agg(pl.first("reported_total_price").alias("total_server_rack_cost"))
)

first_quotes = (
    validated_quotes
    .sort(by="quote_date", descending=False)
    .group_by(["Vendor", "Program"])
    .agg(pl.first("reported_total_price").alias("total_server_rack_cost"))
)

# Step 6: Calculate best-in-class pricing for each program
best_in_class_pricing = (
    reshaped_quote_details
    .group_by(["Program", "Item"])
    .agg(pl.min("Unit_Price").alias("min_price"))
    .group_by("Program")
    .agg(pl.sum("min_price").alias("best_in_class_total_cost"))
)

# Output final tables
print("Component Quantities Table:")
print(component_quantities)

print("Latest Quotes Table:")
print(latest_quotes)

print("First Quotes Table:")
print(first_quotes)

print("Best-in-Class Pricing Table:")
print(best_in_class_pricing)


Component Quantities Table:
shape: (60, 3)
┌─────────┬─────────┬──────────┐
│ Item    ┆ Program ┆ Quantity │
│ ---     ┆ ---     ┆ ---      │
│ str     ┆ str     ┆ i64      │
╞═════════╪═════════╪══════════╡
│ CPU     ┆ A       ┆ 2        │
│ GPU     ┆ A       ┆ 0        │
│ RAM     ┆ A       ┆ 4        │
│ SSD     ┆ A       ┆ 1        │
│ HDD     ┆ A       ┆ 0        │
│ …       ┆ …       ┆ …        │
│ TOR     ┆ D       ┆ 1        │
│ CHASSIS ┆ D       ┆ 1        │
│ SERVERS ┆ E       ┆ 10       │
│ TOR     ┆ E       ┆ 1        │
│ CHASSIS ┆ E       ┆ 1        │
└─────────┴─────────┴──────────┘
Latest Quotes Table:
shape: (0, 3)
┌────────┬─────────┬────────────────────────┐
│ Vendor ┆ Program ┆ total_server_rack_cost │
│ ---    ┆ ---     ┆ ---                    │
│ str    ┆ str     ┆ f64                    │
╞════════╪═════════╪════════════════════════╡
└────────┴─────────┴────────────────────────┘
First Quotes Table:
shape: (0, 3)
┌────────┬─────────┬────────────────────────┐
│ Ven

C:\Users\amitp\AppData\Local\Temp\ipykernel_39776\1789254663.py:44: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  server_quantities = server_components.melt(id_vars=["Item"], variable_name="Program", value_name="Quantity")
C:\Users\amitp\AppData\Local\Temp\ipykernel_39776\1789254663.py:45: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  rack_quantities = rack_components.melt(id_vars=["Item"], variable_name="Program", value_name="Quantity")
C:\Users\amitp\AppData\Local\Temp\ipykernel_39776\1789254663.py:55: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  reshaped_quote_details = valid_quote_details.melt(


In [192]:
component_quantities

shape: (20, 3)
┌──────┬─────────┬──────────┐
│ Item ┆ Program ┆ Quantity │
│ ---  ┆ ---     ┆ ---      │
│ str  ┆ str     ┆ i64      │
╞══════╪═════════╪══════════╡
│ CPU  ┆ A       ┆ 2        │
│ GPU  ┆ A       ┆ 0        │
│ RAM  ┆ A       ┆ 4        │
│ SSD  ┆ A       ┆ 1        │
│ HDD  ┆ A       ┆ 0        │
│ …    ┆ …       ┆ …        │
│ NIC  ┆ B       ┆ 2        │
│ PSU  ┆ B       ┆ 2        │
│ TRAY ┆ B       ┆ 1        │
│ CPU  ┆ C       ┆ 2        │
│ GPU  ┆ C       ┆ 0        │
└──────┴─────────┴──────────┘


In [170]:
# Define quantities per program from server and rack components
program_quantities = {
    "A": {"CPU": 2, "GPU": 0, "RAM": 4, "SSD": 1, "HDD": 0, "MOBO": 1, "NIC": 2, "PSU": 1, "TRAY": 1, "TOR": 1, "CHASSIS": 1},
    "B": {"CPU": 2, "GPU": 4, "RAM": 4, "SSD": 2, "HDD": 0, "MOBO": 1, "NIC": 2, "PSU": 2, "TRAY": 1, "TOR": 2, "CHASSIS": 1},
    "C": {"CPU": 2, "GPU": 0, "RAM": 4, "SSD": 1, "HDD": 0, "MOBO": 1, "NIC": 2, "PSU": 1, "TRAY": 1, "TOR": 1, "CHASSIS": 1},
    "D": {"CPU": 1, "GPU": 0, "RAM": 8, "SSD": 1, "HDD": 20, "MOBO": 1, "NIC": 2, "PSU": 1, "TRAY": 1, "TOR": 1, "CHASSIS": 1},
    "E": {"CPU": 1, "GPU": 2, "RAM": 8, "SSD": 0, "HDD": 20, "MOBO": 1, "NIC": 1, "PSU": 1, "TRAY": 1, "TOR": 1, "CHASSIS": 1}
}

# Map the quantities to each row in valid_quote_details based on Program
adjusted_quote_details = valid_quote_details.with_columns([
    (pl.col("CPU") * pl.lit(program_quantities[pl.col("Program")][component])).alias(f"{component}_adjusted")
    for component in program_quantities["A"].keys()
])

# Recalculate the total adjusted cost
aggregated_adjusted_details = (
    adjusted_quote_details
    .group_by(["Vendor", "Program", "quote_date"])
    .agg([
        pl.sum(f"{component}_adjusted").alias(component) for component in program_quantities["A"].keys()
    ])
    .with_columns(
        sum([pl.col(component) for component in program_quantities["A"].keys()]).alias("adjusted_total_cost")
    )
)

# Join with valid_quote_summary and compare again
comparison_adjusted = (
    aggregated_adjusted_details
    .join(valid_quote_summary, on=["Vendor", "Program", "quote_date"])
    .select([
        "Vendor", "Program", "quote_date",
        pl.col("adjusted_total_cost"),
        pl.col("reported_total_price"),
        (pl.col("adjusted_total_cost") - pl.col("reported_total_price")).alias("cost_difference")
    ])
)

print("Adjusted Comparison of Aggregated vs. Reported Costs:")
print(comparison_adjusted.head(10))


TypeError: unhashable type: 'Expr'

In [160]:
# Display schema and head of each DataFrame for debugging and validation

print("server_components schema:", server_components.schema)
print("server_components head:")
print(server_components.head())

print("\nrack_components schema:", rack_components.schema)
print("rack_components head:")
print(rack_components.head())

print("\nquote_summary schema:", quote_summary.schema)
print("quote_summary head:")
print(quote_summary.head())

print("\nquote_details schema:", quote_details.schema)
print("quote_details head:")
print(quote_details.head())

print("\nvalid_quote_details schema:", valid_quote_details.schema)
print("valid_quote_details head:")
print(valid_quote_details.head())

print("\nvalid_quote_summary schema:", valid_quote_summary.schema)
print("valid_quote_summary head:")
print(valid_quote_summary.head())

print("\naggregated_details schema:", aggregated_details.schema)
print("aggregated_details head:")
print(aggregated_details.head())

print("\nvalidated_quotes schema:", validated_quotes.schema)
print("validated_quotes head:")
print(validated_quotes.head())

print("\ncomponent_quantities schema:", component_quantities.schema)
print("component_quantities head:")
print(component_quantities.head())

print("\nlatest_quotes schema:", latest_quotes.schema)
print("latest_quotes head:")
print(latest_quotes.head())

print("\nfirst_quotes schema:", first_quotes.schema)
print("first_quotes head:")
print(first_quotes.head())

print("\nbest_in_class_pricing schema:", best_in_class_pricing.schema)
print("best_in_class_pricing head:")
print(best_in_class_pricing.head())




server_components schema: Schema([('Item', String), ('Server A', Int64), ('Server B', Int64), ('Server C', Int64), ('Server D', Int64), ('Server E', Int64)])
server_components head:
shape: (5, 6)
┌──────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ Item ┆ Server A ┆ Server B ┆ Server C ┆ Server D ┆ Server E │
│ ---  ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str  ┆ i64      ┆ i64      ┆ i64      ┆ i64      ┆ i64      │
╞══════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ CPU  ┆ 2        ┆ 2        ┆ 2        ┆ 1        ┆ 1        │
│ GPU  ┆ 0        ┆ 4        ┆ 0        ┆ 0        ┆ 2        │
│ RAM  ┆ 4        ┆ 4        ┆ 4        ┆ 8        ┆ 8        │
│ SSD  ┆ 1        ┆ 2        ┆ 1        ┆ 1        ┆ 0        │
│ HDD  ┆ 0        ┆ 0        ┆ 0        ┆ 20       ┆ 20       │
└──────┴──────────┴──────────┴──────────┴──────────┴──────────┘

rack_components schema: Schema([('Item', String), ('Server A', Int64), ('Server B', Int64), ('Serve

In [178]:
best_in_class_pricing

Program,best_in_class_total_cost
str,f64
"""Program_C""",2740.18
"""Program_A""",2839.25
"""Program_E""",2754.7
"""Program_B""",2750.58
"""Program_D""",2758.21


In [179]:
# aggregated_details
valid_quote_summary

Vendor,Program,quote_timestamp,reported_total_price,quote_date
str,str,datetime[μs],f64,date
"""Vendor_7""","""A""",2024-09-08 09:24:26.457788,18166.85,2024-09-08
"""Vendor_7""","""C""",2024-09-04 20:54:28.430814,20561.73,2024-09-04
"""Vendor_1""","""D""",2024-09-24 20:49:52.296317,52527.28,2024-09-24
"""Vendor_1""","""D""",2024-09-04 17:07:58.019610,36545.28,2024-09-04
"""Vendor_5""","""D""",2024-09-04 20:43:51.024192,36864.57,2024-09-04
…,…,…,…,…
"""Vendor_6""","""B""",2024-09-03 04:36:02.445715,29320.38,2024-09-03
"""Vendor_7""","""C""",2024-09-02 00:38:15.661547,22139.59,2024-09-02
"""Vendor_1""","""E""",2024-09-24 10:23:03.302944,54118.35,2024-09-24


In [181]:
latest_quotes
first_quotes

Vendor,Program,total_server_rack_cost
str,str,f64


In [183]:
# Check unique combinations in aggregated_details
print("Unique combinations in Aggregated Details:")
print(aggregated_details.select(["Vendor", "Program", "quote_date"]).unique().head(10))

# Check unique combinations in valid_quote_summary
print("Unique combinations in Valid Quote Summary:")
print(valid_quote_summary.select(["Vendor", "Program", "quote_date"]).unique().head(10))


Unique combinations in Aggregated Details:
shape: (0, 3)
┌────────┬─────────┬────────────┐
│ Vendor ┆ Program ┆ quote_date │
│ ---    ┆ ---     ┆ ---        │
│ str    ┆ str     ┆ date       │
╞════════╪═════════╪════════════╡
└────────┴─────────┴────────────┘
Unique combinations in Valid Quote Summary:
shape: (10, 3)
┌──────────┬─────────┬────────────┐
│ Vendor   ┆ Program ┆ quote_date │
│ ---      ┆ ---     ┆ ---        │
│ str      ┆ str     ┆ date       │
╞══════════╪═════════╪════════════╡
│ Vendor_3 ┆ C       ┆ 2024-09-10 │
│ Vendor_2 ┆ C       ┆ 2024-09-04 │
│ Vendor_1 ┆ A       ┆ 2024-09-22 │
│ Vendor_1 ┆ A       ┆ 2024-09-20 │
│ Vendor_5 ┆ D       ┆ 2024-09-16 │
│ Vendor_3 ┆ C       ┆ 2024-09-12 │
│ Vendor_1 ┆ B       ┆ 2024-09-06 │
│ Vendor_1 ┆ C       ┆ 2024-09-21 │
│ Vendor_6 ┆ D       ┆ 2024-09-02 │
│ Vendor_1 ┆ B       ┆ 2024-09-12 │
└──────────┴─────────┴────────────┘


In [217]:
import polars as pl
from datetime import datetime, timedelta

# Helper function to get the first Monday of a given month
def get_first_monday(year, month):
    first_day = datetime(year, month, 1)
    days_until_monday = (7 - first_day.weekday()) % 7
    return first_day + timedelta(days=days_until_monday)

# Function to filter quotes within valid dates
def fct_valid_dates(df, year, month):
    first_monday = get_first_monday(year, month)
    month_25th = datetime(year, month, 25)
    df_return = (
        df.with_columns(
            pl.col("quote_timestamp").str.to_datetime()
        )
        .filter(
            pl.col("quote_timestamp").is_between(first_monday, month_25th)
        )
    )
    return df_return

# Load data
server_components = pl.read_excel("data/program configurations.xlsx", sheet_name="server_specs")
rack_components = pl.read_excel("data/program configurations.xlsx", sheet_name="rack_specs")
quote_summary = pl.read_csv("data/quote_summaries.csv")
quote_details = pl.read_csv("data/quote_lines.csv")

# Step 1: Calculate extended quantities per component across all programs
extended_quantities = (
    pl.concat([server_components, rack_components], how="vertical")
    .group_by("Item")
    .sum()
)

# Filtered summary and details with valid dates and vendor/program restrictions
valid_programs = ["A", "B", "D"]
valid_vendors = ["Vendor_4", "Vendor_5", "Vendor_6", "Vendor_7"]
# Concatenate vendor and program columns for exclusion
excluded_vendor_programs = {"Vendor_7-C", "Vendor_7-E"}

# Apply date filtering to quote summaries and details
quote_summary = fct_valid_dates(quote_summary, 2024, 9)
quote_details = fct_valid_dates(quote_details, 2024, 9)

# Filtered `quote_summary` for vendor and program restrictions
filtered_summary = (
    quote_summary
    .with_columns(
        pl.concat_str(["vendor", "program"], separator="-").alias("vendor_program")
    )
    .filter(
        pl.col("program").is_in(valid_programs) &
        pl.col("vendor").is_in(valid_vendors) &
        ~pl.col("vendor_program").is_in(excluded_vendor_programs)
    )
    .drop("vendor_program")
)

# Filtered `quote_details` for vendor and program restrictions
filtered_details = (
    quote_details
    .with_columns(
        pl.concat_str(["Vendor", "Program"], separator="-").alias("vendor_program")
    )
    .filter(
        pl.col("Program").is_in(valid_programs) &
        pl.col("Vendor").is_in(valid_vendors) &
        ~pl.col("vendor_program").is_in(excluded_vendor_programs)
    )
    .drop("vendor_program")
)


# Step 3: Calculate total costs per program per vendor
# Latest quote per vendor and program
latest_quote = (
    filtered_summary.sort("quote_timestamp", descending=True)
    .group_by(["vendor", "program"])
    .first()
)

# First quote per vendor and program
first_quote = (
    filtered_summary.sort("quote_timestamp")
    .group_by(["vendor", "program"])
    .first()
)

# Reshape `filtered_details` to have a "Component" and "Unit_Price" column using `unpivot`
melted_details = (
    filtered_details
    .unpivot(
        index=["Vendor", "Program", "quote_timestamp"],  # Columns to keep as index
        variable_name="Component",
        value_name="Unit_Price"
    )
)

# Calculate best-in-class pricing
best_in_class_pricing = (
    melted_details
    .group_by(["Program", "Component"])
    .agg(pl.min("Unit_Price").alias("min_price"))
    .group_by("Program")
    .agg(pl.sum("min_price").alias("best_in_class_total_cost"))
)

# Display final results
print("Extended Quantities per Component for All Programs:")
print(extended_quantities)

print("\nTotal Cost per Program per Vendor (Latest Quote):")
print(latest_quote)

print("\nTotal Cost per Program per Vendor (First Quote):")
print(first_quote)

print("\nBest-in-Class Pricing per Program:")
print(best_in_class_pricing)


Extended Quantities per Component for All Programs:
shape: (12, 6)
┌─────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ Item    ┆ Server A ┆ Server B ┆ Server C ┆ Server D ┆ Server E │
│ ---     ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str     ┆ i64      ┆ i64      ┆ i64      ┆ i64      ┆ i64      │
╞═════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ HDD     ┆ 0        ┆ 0        ┆ 0        ┆ 20       ┆ 20       │
│ CPU     ┆ 2        ┆ 2        ┆ 2        ┆ 1        ┆ 1        │
│ SSD     ┆ 1        ┆ 2        ┆ 1        ┆ 1        ┆ 0        │
│ TRAY    ┆ 1        ┆ 1        ┆ 1        ┆ 1        ┆ 1        │
│ CHASSIS ┆ 1        ┆ 1        ┆ 1        ┆ 1        ┆ 1        │
│ …       ┆ …        ┆ …        ┆ …        ┆ …        ┆ …        │
│ SERVERS ┆ 12       ┆ 8        ┆ 14       ┆ 10       ┆ 10       │
│ TOR     ┆ 1        ┆ 2        ┆ 1        ┆ 1        ┆ 1        │
│ PSU     ┆ 1        ┆ 2        ┆ 1        ┆ 1        ┆ 1     

In [220]:
import polars as pl
from datetime import datetime, timedelta

# Helper function to get the first Monday of a given month
def get_first_monday(year, month):
    first_day = datetime(year, month, 1)
    days_until_monday = (7 - first_day.weekday()) % 7
    return first_day + timedelta(days=days_until_monday)

# Function to filter quotes within valid dates
def fct_valid_dates(df, year, month):
    first_monday = get_first_monday(year, month)
    month_25th = datetime(year, month, 25)
    df_return = (
        df.with_columns(
            pl.col("quote_timestamp").str.to_datetime()
        )
        .filter(
            pl.col("quote_timestamp").is_between(first_monday, month_25th)
        )
    )
    return df_return

# Load data
server_components = pl.read_excel("data/program configurations.xlsx", sheet_name="server_specs")
rack_components = pl.read_excel("data/program configurations.xlsx", sheet_name="rack_specs")
quote_summary = pl.read_csv("data/quote_summaries.csv")
quote_details = pl.read_csv("data/quote_lines.csv")

# Step 1: Calculate extended quantities per component across all programs
extended_quantities = (
    pl.concat([server_components, rack_components], how="vertical")
    .group_by("Item")
    .sum()
)

# Define valid programs and vendors based on original data
valid_programs = ["A", "B", "C", "D", "E"]  # Adjusted to include all relevant programs
valid_vendors = ["Vendor_4", "Vendor_5", "Vendor_6", "Vendor_7"]  # Keep only valid vendors
excluded_vendor_programs = {"Vendor_7-C", "Vendor_7-E"}

# Apply date filtering to quote summaries and details
quote_summary = fct_valid_dates(quote_summary, 2024, 9)
quote_details = fct_valid_dates(quote_details, 2024, 9)

# Filtered `quote_summary` for vendor and program restrictions
filtered_summary = (
    quote_summary
    .with_columns(
        pl.concat_str(["vendor", "program"], separator="-").alias("vendor_program")
    )
    .filter(
        pl.col("program").is_in(valid_programs) &
        pl.col("vendor").is_in(valid_vendors) &
        ~pl.col("vendor_program").is_in(excluded_vendor_programs)
    )
    .drop("vendor_program")
)

# Filtered `quote_details` for vendor and program restrictions
filtered_details = (
    quote_details
    .with_columns(
        pl.concat_str(["Vendor", "Program"], separator="-").alias("vendor_program")
    )
    .filter(
        pl.col("Program").is_in(valid_programs) &
        pl.col("Vendor").is_in(valid_vendors) &
        ~pl.col("vendor_program").is_in(excluded_vendor_programs)
    )
    .drop("vendor_program")
)

# Check filtered_details content
print("Filtered Details Sample After Adjustments:")
print(filtered_details.head())

# Step 3: Calculate total costs per program per vendor
# Latest quote per vendor and program
latest_quote = (
    filtered_summary.sort("quote_timestamp", descending=True)
    .group_by(["vendor", "program"])
    .first()
)

# First quote per vendor and program
first_quote = (
    filtered_summary.sort("quote_timestamp")
    .group_by(["vendor", "program"])
    .first()
)

# Step 4: Calculate "Best-in-Class" Pricing
# Reshape `filtered_details` to have a "Component" and "Unit_Price" column using `unpivot`
melted_details = (
    filtered_details
    .unpivot(
        index=["Vendor", "Program", "quote_timestamp"],  # Columns to keep as index
        variable_name="Component",
        value_name="Unit_Price"
    )
)

# Calculate best-in-class pricing
best_in_class_pricing = (
    melted_details
    .group_by(["Program", "Component"])
    .agg(pl.min("Unit_Price").alias("min_price"))
    .group_by("Program")
    .agg(pl.sum("min_price").alias("best_in_class_total_cost"))
)

# Display final results
print("Extended Quantities per Component for All Programs:")
print(extended_quantities)

print("\nTotal Cost per Program per Vendor (Latest Quote):")
print(latest_quote)

print("\nTotal Cost per Program per Vendor (First Quote):")
print(first_quote)

print("\nBest-in-Class Pricing per Program:")
print(best_in_class_pricing)


Filtered Details Sample After Adjustments:
shape: (0, 14)
┌────────┬─────────┬─────────────────┬─────┬───┬─────┬──────┬─────┬─────────┐
│ Vendor ┆ Program ┆ quote_timestamp ┆ CPU ┆ … ┆ PSU ┆ TRAY ┆ TOR ┆ CHASSIS │
│ ---    ┆ ---     ┆ ---             ┆ --- ┆   ┆ --- ┆ ---  ┆ --- ┆ ---     │
│ str    ┆ str     ┆ datetime[μs]    ┆ f64 ┆   ┆ f64 ┆ f64  ┆ f64 ┆ f64     │
╞════════╪═════════╪═════════════════╪═════╪═══╪═════╪══════╪═════╪═════════╡
└────────┴─────────┴─────────────────┴─────┴───┴─────┴──────┴─────┴─────────┘
Extended Quantities per Component for All Programs:
shape: (12, 6)
┌─────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ Item    ┆ Server A ┆ Server B ┆ Server C ┆ Server D ┆ Server E │
│ ---     ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str     ┆ i64      ┆ i64      ┆ i64      ┆ i64      ┆ i64      │
╞═════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ CHASSIS ┆ 1        ┆ 1        ┆ 1        ┆ 1        ┆ 1        │
│ MOB

In [221]:
import polars as pl
from datetime import datetime, timedelta

# Helper function to get the first Monday of a given month
def get_first_monday(year, month):
    first_day = datetime(year, month, 1)
    days_until_monday = (7 - first_day.weekday()) % 7
    return first_day + timedelta(days=days_until_monday)

# Function to filter quotes within valid dates
def fct_valid_dates(df, year, month):
    first_monday = get_first_monday(year, month)
    month_25th = datetime(year, month, 25)
    df_return = (
        df.with_columns(
            pl.col("quote_timestamp").str.to_datetime()
        )
        .filter(
            pl.col("quote_timestamp").is_between(first_monday, month_25th)
        )
    )
    return df_return

# Load data
server_components = pl.read_excel("data/program configurations.xlsx", sheet_name="server_specs")
rack_components = pl.read_excel("data/program configurations.xlsx", sheet_name="rack_specs")
quote_summary = pl.read_csv("data/quote_summaries.csv")
quote_details = pl.read_csv("data/quote_lines.csv")

# Step 1: Calculate extended quantities per component across all programs
extended_quantities = (
    pl.concat([server_components, rack_components], how="vertical")
    .group_by("Item")
    .sum()
)

# Apply date filtering and program exclusion for quote_details
valid_quote_details = (
    fct_valid_dates(quote_details, year=2024, month=9)
    .filter(~((pl.col("Vendor") == "Vendor_7") & (pl.col("Program").is_in(["Program_C", "Program_E"]))))
)

# Apply date filtering and program exclusion for quote_summary
valid_quote_summary = (
    fct_valid_dates(quote_summary, year=2024, month=9)
    .filter(~((pl.col("vendor") == "Vendor_7") & (pl.col("program").is_in(["Program_C", "Program_E"]))))
    .rename({"vendor": "Vendor", "program": "Program"})
)

# Check the filtered datasets
print("Filtered Quote Details Sample:")
print(valid_quote_details.head())

print("Filtered Quote Summary Sample:")
print(valid_quote_summary.head())

# Step 3: Calculate total costs per program per vendor
# Latest quote per vendor and program
latest_quote = (
    valid_quote_summary.sort("quote_timestamp", descending=True)
    .group_by(["Vendor", "Program"])
    .first()
)

# First quote per vendor and program
first_quote = (
    valid_quote_summary.sort("quote_timestamp")
    .group_by(["Vendor", "Program"])
    .first()
)

# Step 4: Calculate "Best-in-Class" Pricing
# Reshape `valid_quote_details` to have a "Component" and "Unit_Price" column using `unpivot`
melted_details = (
    valid_quote_details
    .unpivot(
        index=["Vendor", "Program", "quote_timestamp"],  # Columns to keep as index
        variable_name="Component",
        value_name="Unit_Price"
    )
)

# Calculate best-in-class pricing
best_in_class_pricing = (
    melted_details
    .group_by(["Program", "Component"])
    .agg(pl.min("Unit_Price").alias("min_price"))
    .group_by("Program")
    .agg(pl.sum("min_price").alias("best_in_class_total_cost"))
)

# Display final results
print("Extended Quantities per Component for All Programs:")
print(extended_quantities)

print("\nTotal Cost per Program per Vendor (Latest Quote):")
print(latest_quote)

print("\nTotal Cost per Program per Vendor (First Quote):")
print(first_quote)

print("\nBest-in-Class Pricing per Program:")
print(best_in_class_pricing)


Filtered Quote Details Sample:
shape: (5, 14)
┌──────────┬───────────┬─────────────────┬────────┬───┬────────┬───────┬────────┬─────────┐
│ Vendor   ┆ Program   ┆ quote_timestamp ┆ CPU    ┆ … ┆ PSU    ┆ TRAY  ┆ TOR    ┆ CHASSIS │
│ ---      ┆ ---       ┆ ---             ┆ ---    ┆   ┆ ---    ┆ ---   ┆ ---    ┆ ---     │
│ str      ┆ str       ┆ datetime[μs]    ┆ f64    ┆   ┆ f64    ┆ f64   ┆ f64    ┆ f64     │
╞══════════╪═══════════╪═════════════════╪════════╪═══╪════════╪═══════╪════════╪═════════╡
│ Vendor_6 ┆ Program_E ┆ 2024-09-12      ┆ 324.21 ┆ … ┆ 89.99  ┆ 70.05 ┆ 728.08 ┆ 1068.57 │
│          ┆           ┆ 00:26:22.414219 ┆        ┆   ┆        ┆       ┆        ┆         │
│ Vendor_5 ┆ Program_A ┆ 2024-09-23      ┆ 300.29 ┆ … ┆ 102.29 ┆ 22.98 ┆ 734.33 ┆ 1770.98 │
│          ┆           ┆ 18:35:08.078890 ┆        ┆   ┆        ┆       ┆        ┆         │
│ Vendor_5 ┆ Program_B ┆ 2024-09-24      ┆ 333.53 ┆ … ┆ 113.9  ┆ 80.13 ┆ 616.13 ┆ 1563.11 │
│          ┆           ┆ 11:33:41.